In [ ]:
from autumn.infrastructure.remote import springboard

In [ ]:
from datetime import datetime
import numpy as np

In [ ]:
from summer2 import CompartmentalModel
from summer2.parameters import Parameter as param

In [ ]:
def get_model():
    m = CompartmentalModel([0,100], ["S", "I", "R"], "I", ref_date=datetime(2001,1,1))
    m.set_initial_population({"S": 990.0, "I": 10.0})
    m.add_infection_frequency_flow("infection", param("contact_rate"), "S", "I")
    m.add_transition_flow("recovery", param("recovery_rate"), "I", "R")
    incidence = m.request_output_for_flow("incidence", "infection")
    m.request_function_output("notifications", incidence * param("cdr"))
    m.set_default_parameters({"contact_rate": 0.4, "recovery_rate": 0.1, "cdr": 0.2})
    return m


In [ ]:
from estival import targets as est
from estival import priors as esp
from estival.model import BayesianCompartmentalModel

In [ ]:
# We're going to run multiple calibrations in parallel - to verify that they are doing different and meaningful work,
# we're going to generate different targets based on running the model with a range of contact_rates for the synthetic data

contact_rates = np.linspace(0.3,0.9,4)
contact_rates

In [ ]:
# Our parameterised targets function
# build a model so that we've got something to run it with
m = get_model()

def get_targets(contact_rate):
    m.run({"contact_rate": contact_rate, "recovery_rate": 0.4})
    do_def = m.get_derived_outputs_df()
    obs_clean = do_def["incidence"].iloc[0:50]
    obs_noisy = obs_clean * np.exp(np.random.normal(0.0,0.2,len(obs_clean)))
    targets = [
    est.TruncatedNormalTarget("incidence", obs_noisy, (0.0,np.inf),
        esp.UniformPrior("incidence_dispersion",(0.1, obs_noisy.max()*0.1)))
    ]
    return targets

In [ ]:
# Uniform priors over our 2 model parameters
priors = [
    esp.UniformPrior("contact_rate", (0.01,1.0)),
    esp.TruncNormalPrior("recovery_rate", 0.5, 0.2, (0.01,1.0)),
]

In [ ]:
from estival.calibration import pymc as epm
import pymc as pm
import arviz as az

In [ ]:
def calibrate_model(targets, priors, draws: int, chains: int):
    
    # Build our model as specified above
    m = get_model()
    defp = m.get_default_parameters()
    
    # Build the BCM based on user supplied targets and priors
    bcm = BayesianCompartmentalModel(m, defp, priors, targets)
    
    # Run for the specified number of draws, over the specified number of chains/cores
    with pm.Model() as model:
        variables = epm.use_model(bcm)
        idata = pm.sample(step=[pm.DEMetropolis(variables)], draws=draws, tune=0,cores=chains,chains=chains)
        
    return idata

In [ ]:
def remote_calibration_task(bridge: springboard.task.TaskBridge, targets, priors, draws, chains):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')
    
    bridge.logger.info(f"Calibrating {chains} chains for {draws} draws")
    
    idata = calibrate_model(targets, priors, draws, chains)
    idata.to_netcdf(bridge.out_path / "idata.nc")

    summary = az.summary(idata)
    bridge.logger.info(summary["r_hat"])
    
    bridge.logger.info("Calibration complete")

In [ ]:
N_CHAINS = 4

In [ ]:
# Get a compute machine to do.. some computation.
# We only use a single EC2MachineSpec - all instances will be of the same specification
mspec = springboard.EC2MachineSpec(N_CHAINS, 4, "compute")

In [ ]:
#custom run_path function
def generate_run_paths(n_runs: int, desc: str):
    base_name = springboard.launch.gen_run_name(desc)
    return [f"projects/testing/somewhere/{base_name}/run{i}" for i in range(n_runs)]
    

In [ ]:
run_paths = generate_run_paths(4, "multical")
run_paths

In [ ]:
#launch_synced_multiple_autumn_task operates in a similar fashion to launch_synced_autumn_task
#however instead of a single TaskSpec and run_path, it takes a dictionary of form
#{run_path: task_spec}

task_dict = {}

for i, cr in enumerate(contact_rates):
    # Use our custom targets function from above
    targets = get_targets(cr)
    # Use these targets, but keep the other kwargs the same
    task_kwargs = {
        "targets": targets,
        "priors": priors,
        "draws": 2000,
        "chains": N_CHAINS
    }
    task_dict[run_paths[i]] = springboard.TaskSpec(remote_calibration_task, task_kwargs)

In [ ]:
runners = springboard.launch.launch_synced_multiple_autumn_task(task_dict, mspec)

In [ ]:
for k, runner in runners.items():
    print(runner.wait())

In [ ]:
from autumn.core.runs import ManagedRun

In [ ]:
mr_dict = {}

for run_path in run_paths:
    mr_dict[run_path] = mr = ManagedRun(run_path)
    for f in mr.remote.list_contents():
        mr.remote.download(f)

In [ ]:
import arviz as az

In [ ]:
# Select a run here, see results below
run_path = run_paths[0]
mr = mr_dict[run_path]
run_path

In [ ]:
idata = az.from_netcdf(mr.local_path / "output/idata.nc")

In [ ]:
az.summary(idata)

In [ ]:
az.plot_trace(idata, compact=False);